In [1]:
push!( LOAD_PATH, "." ); # folder que continen los modulos. Correr desde ahí dentro.
#
using Z_auxiliaresBRW, Z_auxiliaresGEN
using HDF5

In [2]:
arxname="/home/karel/BRWFiles/Isabel2019/control_02.brw"

"/home/karel/BRWFiles/Isabel2019/control_02.brw"

In [3]:
pmem=filesize(arxname)/Sys.total_memory()

0.5216578024702109

In [55]:
function guardadictydatos(oldie::Dict, nuevos::Array, nombre::String)
    llaves=keys(oldie)
    
    if in( "dset", llaves)
        
        oldie["dset"]=nuevos
        println("guardando en $nombre")
        
        h5open(nombre, "w")  do file
            for k in llaves
                datos=oldie[k]
                write(file, "$k", datos)
            end
        end
        return
    else   
        error("Tu dict no trae dataset a remplazar")
    end
        
end

guardadictydatos (generic function with 1 method)

In [61]:
dirname(arxname), basename(arxname)

("/home/karel/BRWFiles/Isabel2019", "control_02.brw")

In [77]:
function cortayguarda(filenombre::String, pmem=0.25)
    
    
    if pmem<0.25 #pmem es memoria usada, no libre!!
        println("tu archivo tiene tamaño razonable, dejalo asi")
        return 0
    else

    dicth5 = brw_things( filenombre ); # variables útiles
    data = dicth5[ "dset" ]; # path al dataset (NO READ! to much)
    M=size(data, 1) # el tamaño de data
    chans=dicth5["Chs"]
    nch=size(chans, 1)
    nframes=Int(M/nch)
    
    
    ncachos=max(ceil(Int, 1/(1-pmem)), 4) # al menos partela en 4
    fcacho=floor(Int, nframes/ncachos) # frames por cacho dara un poco menos que el total.
        
    paso=nch*fcacho #cuantos datos tiene cada cacho.
        
    println("Cuadros totales: \t", nframes, "\n", 
            "Datos totales: \t",  M, "\n", 
            "Cachos: \t " ,ncachos, "\n",
            "Cuadros por cacho: \t", fcacho, "\n")
    
    
    for n = 1:ncachos  # numero de Β a cortar (1-> 4096,1->ω)
        
        cini=(n-1)*fcacho+1
        cfin=n*fcacho
        datitos =reshape(data[((n-1)*paso+1):n*paso], nch, fcacho)
        
        nstring=lpad(n, 2, "0")
        dir=dirname(filenombre)
        fn=basename(filenombre)
        nuname=dir*"/Cacho_"*nstring*"_"*fn
        
            
            
        notacacho="Este es un intervalo de un dataset mas grande. \n
                    Contiene los cuadros de $cini a $cfin."
        dicth5["notacacho"]=notacacho
            
        guardadictydatos(dicth5,datitos, nuname)
        
        println( "vamos en el cacho", n,  " de ", ncachos, " que va del cuadro ",
                cini, " al ",  n*cfin, " guardado en $nuname")
    end
    print("bla")
    return  "Chido"
    end

end

cortayguarda (generic function with 2 methods)

In [ ]:
cortayguarda(arxname, pmem)

Cuadros totales: 	3213990
Datos totales: 	13164503040
Cachos: 	 4
Cuadros por cacho: 	803497

guardando en /home/karel/BRWFiles/Isabel2019/Cacho_01_control_02.brw
vamos en el cacho1 de 4 que va del cuadro 1 al 803497 guardado en /home/karel/BRWFiles/Isabel2019/Cacho_01_control_02.brw
guardando en /home/karel/BRWFiles/Isabel2019/Cacho_02_control_02.brw
vamos en el cacho